## Dataset Sort



Software License Agreement (BSD)

File:annotation_gen.py
Author: Amir Hossein Ebrahimnezhad ebrahimnezhad@ualberta.ca
Maintainer: Martin Barczyk mbarczyk@ualberta.ca
Copyright: Copyright (c) 2023, Mechatronic Systems Lab (University of Alberta), All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that
the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the
following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the
following disclaimer in the documentation and/or other materials provided with the distribution.
Neither the name of Mechatronic Systems Lab nor the names of its contributors may be used to endorse or promote
products derived from this software without specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WAR-
RANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, IN-
DIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT
OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


___________________________________________________________________________________________________________________________

This code generates the required annotation files as .json format for torchvision KRCNN train.

In [4]:
import pandas as pd
import numpy as np

import json
import os

import shutil

import rospkg


In [5]:
rospack = rospkg.RosPack()
workspace_dir = rospack.get_path('bebop_ros')	

dir_path = workspace_dir + "/repo/dataset"


In [6]:
# data_num = 7200
data_num = 10
n_train = int(data_num*0.9)
n_test = int(data_num*0.1)


csv_data = pd.read_csv(workspace_dir + '/repo/dataset/csv/img_rel_pose.csv')
csv_np = csv_data.to_numpy()

In [7]:
path_annot = workspace_dir + '/repo/rcnn'


In [8]:
try:
    shutil.rmtree(os.path.join(path_annot, "train"))
except:
    pass

try:
    shutil.rmtree(os.path.join(path_annot, "test"))	
except:
    pass



os.mkdir(os.path.join(path_annot, "train"))
os.mkdir(os.path.join(path_annot, "test"))

os.mkdir(os.path.join(path_annot, "train", "annotations"))
os.mkdir(os.path.join(path_annot, "train", "images"))

os.mkdir(os.path.join(path_annot, "test", "annotations"))
os.mkdir(os.path.join(path_annot, "test", "images"))


In [9]:
path_img = workspace_dir + "/repo/dataset/data/frames_selected"

img_list = os.listdir(path_img)
img_list.sort()

img_list  = csv_np[:, 0]

img_idx = np.random.permutation(len(img_list))
img_idx = img_idx[:data_num]

idx_train = img_idx[:n_train]
idx_test = img_idx[n_train:]

annot_dict = {"folders": None, "idx": None}

annot_dict["folders"] = ["train", "test"]
annot_dict["idx"] = {"train": idx_train, "test": idx_test}
annot_dict["num"] = {"train": n_train, "test": n_test}


In [10]:
def find(array, item):
    for idx, val in np.ndenumerate(array):
        if val == item:
            return idx
    # If no item was found return None, other return types might be a problem due to
    # numbas type inference.
 

In [11]:
for folder in annot_dict["folders"]:

    idx_i = annot_dict["idx"][folder]

    for i in range(annot_dict["num"][folder]):


        json_file = csv_np[idx_i[i], 0][:-3]+"json"

        annot_temp = {"bboxes": None, "keypoints": None}
        kp_num = 8

        bbox = []
        keypoints = [[[0, 0, 0]]*kp_num]

        
        for j in range(kp_num):

            keypoints[0][j] = [csv_np[idx_i[i], 5+2*j], csv_np[idx_i[i], 6+2*j], 1]
            


        bbox.append([csv_np[idx_i[i], -4], csv_np[idx_i[i], -3], csv_np[idx_i[i], -2], csv_np[idx_i[i],-1]])



        annot_temp['bboxes'] = bbox
        annot_temp['keypoints'] = [keypoints]

        img_src = os.path.join(path_img, csv_np[idx_i[i], 0])
        img_dst = os.path.join(path_annot, folder, "images", csv_np[idx_i[i], 0])

            
        shutil.copy(img_src, img_dst)

        with open(os.path.join(path_annot, folder, "annotations", json_file), 'w') as file:
            json.dump(annot_temp, file)

